# 前処理可視化ノート
前処理モジュール `src/utils/preprocessing.py` で実装した各種処理の効果を確認するためのデモです。
ハイパーパラメータは `config.yaml` から読み込みます。

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from src.utils import preprocessing
from src.utils.config_utils import get_preprocessing_params

params = get_preprocessing_params()
window_size = params.get('window_size', 256)
stride = params.get('stride', 128)

t = np.linspace(0, window_size/params['sampling_rate'], window_size)
acc_x = np.sin(2*np.pi*1*t)
acc_y = np.cos(2*np.pi*1*t)
acc_z = np.sin(2*np.pi*0.5*t)
quat = np.array([np.ones_like(t), np.zeros_like(t), np.zeros_like(t), np.zeros_like(t)]).T
df = pd.DataFrame({
    'acc_x': acc_x,
    'acc_y': acc_y,
    'acc_z': acc_z,
    'rot_w': quat[:,0],
    'rot_x': quat[:,1],
    'rot_y': quat[:,2],
    'rot_z': quat[:,3],
    'handedness': 1,
    'gesture': 0,
    'subject': 0,
    'sequence_id': 0,
})

In [ ]:
# 線形加速度の計算例
lin_acc = np.array([
    preprocessing.linear_acceleration(row[['acc_x','acc_y','acc_z']].values,
                                      row[['rot_w','rot_x','rot_y','rot_z']].values)
    for _, row in df.iterrows()
])
df[['lin_acc_x','lin_acc_y','lin_acc_z']] = lin_acc

# FFT バンドエネルギー
bands = params['fft_bands']
window = df[['lin_acc_x','lin_acc_y','lin_acc_z']].values[np.newaxis, :, :]
fft_energy = preprocessing.compute_fft_band_energy(window, fs=params['sampling_rate'], bands=bands)
fft_energy

In [ ]:
plt.figure(figsize=(6,3))
plt.plot(t, df['acc_x'], label='acc_x')
plt.plot(t, df['lin_acc_x'], label='lin_acc_x')
plt.legend()
plt.title('Linear Acceleration Example')
plt.show()